In [8]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

In [9]:
working_dir = '/home/Work/clusters/'
all_craters = pd.read_csv(os.path.join(working_dir,'all_data_clusters.csv'))

f = os.path.join(working_dir, 'ndarray_of_all_craters.txt')

files = np.loadtxt(os.path.join(working_dir, 'filename_list.txt'), dtype='str')
total_f = len(files)
total_c = 3


data_all = np.loadtxt(f).reshape((total_f, total_c, 1100))
print(data_all.shape)

(709, 3, 1100)


In [10]:
# create 11 bins of root-2 

k=1
bins=[k]
for i in range(11):
    k*= 2**(0.5)
    bins.append(k)

mid_bin = []
bin_width = []
for i in range(len(bins)-1):
    mid_bin.append( (bins[i] + (bins[i+1]-bins[i]) / 2  ) )
    bin_width.append(bins[i+1]-bins[i])

bin_width = np.array(bin_width)
mid_bin = np.array(mid_bin)


# interactive plot for clusters with a given number of craters

In [ ]:
plt.close()


def interact_plot(N, upper_limit, lower_limit, remove_start):#, remove_end):
    plt.figure(figsize=(8,6))
    dsfd_values=np.empty([len(bins)-1, total_f])*np.nan
    count=0
    color=iter(plt.cm.gist_ncar(np.linspace(0,1,N)))

    for i in range(total_f):
        # get data for one cluster
        p_data = np.array([n for n in data_all[i, :,:][0] if n>0.005])
        
        # if there is less/more than a certain number of craters in the cluster, skip it
        if len(p_data)>lower_limit and count<N and len(p_data)<upper_limit:
            
           # use same colour per loop
            c=next(color)
           # collect details (best fit ellipse area) from master sheet about the crater. try is needed in case not found
            try:
                area = all_craters.loc[all_craters["Observation_ID"]== files[i], "area_ellipse"].iloc[0]
                area /=1e6
            except IndexError:
                print(f'the crater {files[i]} was not found in the all_craters csv file. using an area of 1 km2')
                area = 1/1e6
            #---------
            # binning and plotting data
            
            # n is the number of clusters per bin
            n = np.array([np.sum((p_data>bins[i])&((p_data<=bins[i+1]))) for i in range(len(bins)-1)])
            
            # normalisation using binwidth and best fit ellipse cluster area
            norm_factor = bin_width * area
            d_vals = n/norm_factor
            # plot values at the mid point of the bin
            plt.plot( mid_bin, d_vals, '.', label=files[i], c=c)
            
            #--------------
            # plot error bars
            stds = n**0.5
            min_n = n - stds
            max_n = n + stds
            
            plt.vlines(mid_bin,min_n/norm_factor,max_n/norm_factor, color=c, alpha = 0.1)


            #---------
            # line fitting
            
            # take only bins with craters over a certain value
            reduced_n_mask = (n>3)     
            
            # removing first bin points as given by interact inputs
            for r in range(remove_start):
                reduced_n_mask[r] = False    
                
            # removing last bin points as given by interact inputs

#             k = 0
#             r = 0
#             while r<remove_end: 
#                 print(r, k, reduced_n_mask[-k])
#                 if reduced_n_mask[-k] == True:    # removing end points where not already false
#                     reduced_n_mask[-k] = False
#                     r+=1
#                 k+=1
#             print(reduced_n_mask)

            # simple linear fit in log space
            #TODO: least squares fit with counting uncertainties taken into account
            log_x_data = np.log10(mid_bin[reduced_n_mask])
            log_y_data = np.log10(d_vals[reduced_n_mask])

            a, b = np.polyfit(log_x_data, log_y_data, 1)
            y = 10**(a * np.log10(mid_bin) + b)

            # plotting fit lines
            plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}', c=c)
            count+=1



    plt.xscale("log")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid()

print("To use this interact tool:\nDetermin the number of clusters you wish to plot (N) \nthat contain a subset of craters within the upper and lower limit values.\nThe remove from start option allow you to adjust the linear fits.\nNote that any bins with less than 3 craters are already removed from the fit")
interact(interact_plot, N=[1, 5, 10, 1000], upper_limit=(5, 600, 10), lower_limit=(5, 400, 10), remove_start=[1, 0, 2, 3, 4, 5])#, remove_end=[1, 0, 2, 3, 4, 5])

# interactive plot for one specified crater

In [ ]:
plt.close()


def interact_plot(cluster, remove_start, remove_end):
    plt.figure(figsize=(8,6))
    dsfd_values=np.empty([len(bins)-1, total_f])*np.nan
    count=0
    color=iter(plt.cm.gist_ncar(np.linspace(0,1,2)))
    
    #---------
    # find desired crater in list
    i = np.where(files == cluster)[0][0]
    
    # get data fror that crater
    p_data = np.array([n for n in data_all[i, :,:][0] if n>0.005])
    print(f'\nThis cluster has {len(p_data)} craters\n.')

    c=next(color)

    # collect details (best fit ellipse area) from master sheet about the crater. try is needed in case not found
    try:
        area = all_craters.loc[all_craters["Observation_ID"]== files[i], "area_ellipse"].iloc[0]
        area /=1e6
    except IndexError:
        print(f'the crater {files[i]} was not found in the all_craters csv file. using an area of 1 km2')
        area = 1/1e6
    
    #---------
    # binning and plotting data

    # n is the number of clusters per bin
    n = np.array([np.sum((p_data>bins[i])&((p_data<=bins[i+1]))) for i in range(len(bins)-1)])

    # normalisation using binwidth and best fit ellipse cluster area
    norm_factor = bin_width * area
    d_vals = n/norm_factor
    # plot values at the mid point of the bin
    plt.plot( mid_bin, d_vals, '.', label=files[i], c=c)

    #--------------
    # plot error bars
    stds = n**0.5
    min_n = n - stds
    max_n = n + stds

    plt.vlines(mid_bin,min_n/norm_factor,max_n/norm_factor, color=c, alpha = 0.1)


    #---------
    # line fitting

    # here we take all bins with at least one crater
    reduced_n_mask = (n>0)     

    # removing first bin points as given by interact inputs
    for r in range(remove_start):
        reduced_n_mask[r] = False    

    # removing last bin points as given by interact inputs, given it is not alreay false
    k = 1
    r = 0
    while r<remove_end: 
        if reduced_n_mask[-k] == True:   
            reduced_n_mask[-k] = False
            r+=1
        k+=1
    
    # simple linear fit in log space
    #TODO: least squares fit with counting uncertainties taken into account
    
    # warn user if there is not enough data/poor data for fit
    if len(mid_bin[reduced_n_mask]) < 0:
        print('all points have been fremoved from fit ')
    elif len(mid_bin[reduced_n_mask]) < 3:
        print('fiting is being performed on two points, and may be poor')
        try:
            log_x_data = np.log10(mid_bin[reduced_n_mask])
            log_y_data = np.log10(d_vals[reduced_n_mask])

            a, b = np.polyfit(log_x_data, log_y_data, 1)
            y = 10**(a * np.log10(mid_bin) + b)


            plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}', c=c)
        except TypeError:
            print('tried and failed with so few points')
    else:
        print(f'fitting is being done on {len(mid_bin[reduced_n_mask])} points')

        log_x_data = np.log10(mid_bin[reduced_n_mask])
        log_y_data = np.log10(d_vals[reduced_n_mask])

        a, b = np.polyfit(log_x_data, log_y_data, 1)
        y = 10**(a * np.log10(mid_bin) + b)

   
        plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}', c=c)



    plt.xscale("log")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid()

print("To use this interact tool:\nSelect a cluster ID.\nThe remove from start/end option allow you to adjust the linear fits.\nFor this plot, all points are included by default")
interact(interact_plot, cluster=files, remove_start=[0, 1, 2, 3, 4, 5], remove_end=[0, 1, 2, 3, 4, 5])

# same again, but without interact

In [ ]:
plt.close()
plt.figure(figsize=(8,6))
dsfd_values=np.empty([len(bins)-1, total_f])*np.nan

limit = 100
count=0
color=iter(plt.cm.gist_ncar(np.linspace(0,1,25)))

for i in range(total_f):
    # get data for one cluster
    p_data = np.array([n for n in data_all[i, :,:][0] if n>0.005])

    # if there is less/more than a certain number of craters in the cluster, skip it
    if len(p_data)>limit:

       # use same colour per loop
        c=next(color)
       # collect details (best fit ellipse area) from master sheet about the crater. try is needed in case not found
        try:
            area = all_craters.loc[all_craters["Observation_ID"]== files[i], "area_ellipse"].iloc[0]
            area /=1e6
        except IndexError:
            print(f'the crater {files[i]} was not found in the all_craters csv file. using an area of 1 km2')
            area = 1/1e6
        #---------
        # binning and plotting data

        # n is the number of clusters per bin
        n = np.array([np.sum((p_data>bins[i])&((p_data<=bins[i+1]))) for i in range(len(bins)-1)])

        # normalisation using binwidth and best fit ellipse cluster area
        norm_factor = bin_width * area
        d_vals = n/norm_factor
        # plot values at the mid point of the bin
        plt.plot( mid_bin, d_vals, '.', label=files[i], c=c)

        #--------------
        # plot error bars
        stds = n**0.5
        min_n = n - stds
        max_n = n + stds

        plt.vlines(mid_bin,min_n/norm_factor,max_n/norm_factor, color=c, alpha = 0.1)


        #---------
        # line fitting

        # take only bins with craters over a certain value
        reduced_n_mask = (n>3)    
        
        # removing first bin points where <1m is unreliable
        reduced_n_mask[0] = False    

        # simple linear fit in log space
        #TODO: least squares fit with counting uncertainties taken into account
        log_x_data = np.log10(mid_bin[reduced_n_mask])
        log_y_data = np.log10(d_vals[reduced_n_mask])

        a, b = np.polyfit(log_x_data, log_y_data, 1)
        y = 10**(a * np.log10(mid_bin) + b)

        # plotting fit lines
        plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}', c=c)
        count+=1


print(f'there are {count} craters above {limit}')
plt.xscale("log")
plt.yscale("log")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()


# for data creation:

In [ ]:
working_dir = '/home/Work/clusters/'
all_craters = pd.read_csv(os.path.join(working_dir,'all_data_clusters.csv'))

# list all  files in this directory
all_files = glob.glob(os.path.join(working_dir,"data/*.xlsx"))

filenames = [os.path.basename(x)[:-5] for x in all_files]
total_f = len(filenames)
total_c = 3

with open(os.path.join(working_dir, 'filename_list.txt'), 'w') as outfile:
    for i in files:
        outfile.write(f'{i}\n')

        
data_all = np.empty([total_f, total_c, 1100])
data_all[:] = np.NaN

for i in range(total_f):
    data = pd.read_excel(all_files[i])
    data.sort_values(data.columns[0])
    print(filenames[i], data)
    k=0
    for j in range(len(data.columns)): 
        col = data.columns[j]
#     tab = Table.from_pandas(data)
        if "oord" in col or "iam" in col:
            data_all[i, k, :len(data[col])]= data[col]
            k+=1

## data_all in format: # data_all = 'Diam_m','long','lat')


In [ ]:
d